In [8]:
# 安装必要的依赖库
!pip install -q mlflow rouge_score transformers datasets evaluate
!pip install evaluate


# Import necessary toolkits
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback
from datasets import Dataset
import pandas as pd
import numpy as np
import torch
from evaluate import load as load_metric
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# 检查GPU是否可用
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [2]:
def preprocess_data(csv_files):
    data = []
    for file in csv_files:
        df = pd.read_csv(file)
        # 根据实际列名设置输入列和标签列
        if 'text' in df.columns:
            text_column = 'text'
        elif 'sentence' in df.columns:
            text_column = 'sentence'
        elif 'prompt' in df.columns:
            text_column = 'prompt'
        else:
            raise ValueError(f"Column for text input not found in {file}. Available columns are: {df.columns}")

        if 'label' not in df.columns:
            raise ValueError(f"Column 'label' not found in {file}. Available columns are: {df.columns}")

        for _, row in df.iterrows():
            text = f"text: {row[text_column]} </s>"
            label = str(row['label'])  # 将标签转换为字符串形式
            data.append({"input_text": text, "target_text": label})
    return data


csv_files = [
    'superset_train.csv'
]
all_data = preprocess_data(csv_files)

In [3]:
# Step 3: 创建数据集
all_data = preprocess_data(csv_files)
dataset = Dataset.from_pandas(pd.DataFrame(all_data))
dataset = dataset.train_test_split(test_size=0.1)
train_data = dataset['train']
validation_data = dataset['test']

In [4]:
# Step 4: 加载T5模型和分词器，并将模型加载到GPU上
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)


# 数据集格式转换为模型输入
def tokenize_data(batch):
    tokenized_input = tokenizer(batch['input_text'], padding='max_length', truncation=True, max_length=512)
    tokenized_label = tokenizer(batch['target_text'], padding='max_length', truncation=True, max_length=64)

    tokenized_input['labels'] = tokenized_label['input_ids']
    return tokenized_input

train_data = train_data.map(tokenize_data, batched=True)
validation_data = validation_data.map(tokenize_data, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


...

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/26725 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Map:   0%|          | 0/2970 [00:00<?, ? examples/s]

In [5]:
BATCH_SIZE = 8
NUM_EPOCHS = 3
LEARNING_RATE = 5e-5

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    logging_dir='./logs',
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    overwrite_output_dir=True,
    save_total_limit=3,
    load_best_model_at_end=True,
    fp16=True,
    learning_rate=LEARNING_RATE,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
# 加载分类评估指标（例如 accuracy）
accuracy_metric = load_metric("accuracy")

# 定义评估函数
def compute_metrics(pred):
    pred_str = tokenizer.batch_decode(pred.predictions, skip_special_tokens=True)
    labels_ids = pred.label_ids
    labels_ids = np.where(labels_ids == -100, tokenizer.pad_token_id, labels_ids)
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    # 转换预测和标签为整数形式
    pred_int = [int(p) for p in pred_str]
    label_int = [int(l) for l in label_str]

    # 计算混淆矩阵和分类指标
    tn, fp, fn, tp = confusion_matrix(label_int, pred_int).ravel()
    accuracy = accuracy_score(label_int, pred_int)
    precision = precision_score(label_int, pred_int, zero_division=0)
    recall = recall_score(label_int, pred_int, zero_division=0)

    # 返回所有需要的信息
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    }


In [9]:
# Step 5: 初始化Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=validation_data,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [10]:
# 开始训练
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,Tp,Tn,Fp,Fn
1,0.007500,0.008335,0.711111,0.633609,0.437262,460,1652,266,592
2,0.007500,0.007431,0.764310,0.665414,0.673004,708,1562,356,344
3,0.007400,0.007345,0.769360,0.653814,0.741445,780,1505,413,272


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=10023, training_loss=0.03720107315821609, metrics={'train_runtime': 2015.9038, 'train_samples_per_second': 39.771, 'train_steps_per_second': 4.972, 'total_flos': 1.08510289330176e+16, 'train_loss': 0.03720107315821609, 'epoch': 3.0})

In [11]:
# 评估模型
eval_results = trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [12]:
eval_results

{'eval_loss': 0.007345444522798061,
 'eval_accuracy': 0.7693602693602694,
 'eval_precision': 0.6538139145012574,
 'eval_recall': 0.7414448669201521,
 'eval_tp': 780,
 'eval_tn': 1505,
 'eval_fp': 413,
 'eval_fn': 272,
 'eval_runtime': 75.5325,
 'eval_samples_per_second': 39.321,
 'eval_steps_per_second': 4.925,
 'epoch': 3.0}